In [1]:
import json
import importlib
import os
current_dir=os.getcwd()
os.chdir("../../")
# print(os.getcwd())
import dist_utilities as dist_ult
# import noise_est_utilities as ne_ult
importlib.reload(dist_ult)
# importlib.reload(ne_ult)
os.chdir(current_dir)
import pickle
from qiskit.result import Result
from qiskit.visualization import plot_histogram, plot_distribution
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
from qiskit.result import QuasiDistribution

In [2]:
with open("ideal_counts_sim.json", "r") as f:
    ideal_counts=json.load(f)
with open("noisy_counts_sim.json", "r") as f:
    noisy_counts=json.load(f)
with open("noisy_noise_est_dist_sim.json", "r") as f:
    noisy_noise_est_dist=json.load(f)
with open("ideal_noise_est_counts_sim.json", "r") as f:
    ideal_noise_est_counts=json.load(f)
print("ideal: ", ideal_noise_est_counts)
print("noise est dist: ", noisy_noise_est_dist)
print("noisy counts: ", noisy_counts)
print("ideal counts: ", ideal_counts)
col_label=list(dict(ideal_noise_est_counts).keys())[0]
# corrected_counts=ne_ult.get_corrected_counts_from_counts(noisy_counts, noisy_noise_est_dist, list(ideal_noise_est_counts.keys())[0])

# print(corrected_counts)

ideal:  {'11111': 200000}
noise est dist:  {'11111': 0.13557, '10011': 0.02979, '01011': 0.036365, '00011': 0.017905, '11011': 0.05885, '10001': 0.01785, '11010': 0.03333, '10000': 0.01215, '11101': 0.059655, '10100': 0.01832, '00010': 0.011195, '01101': 0.032555, '11100': 0.03652, '00110': 0.01716, '10110': 0.031045, '00001': 0.011355, '01000': 0.01301, '01111': 0.058475, '10111': 0.05342, '11110': 0.06075, '10101': 0.02986, '00101': 0.017255, '01001': 0.01994, '00111': 0.027985, '01010': 0.02001, '11000': 0.020555, '11001': 0.03243, '10010': 0.017905, '00100': 0.011105, '01110': 0.03076, '01100': 0.019575, '00000': 0.00735}
noisy counts:  {'11011': 7373, '01100': 5666, '01111': 7868, '00110': 5020, '11111': 19376, '01000': 5616, '11001': 6319, '00100': 4878, '00111': 5621, '11101': 7544, '10111': 6342, '00000': 4787, '00010': 4972, '01011': 6091, '00011': 5117, '11000': 6113, '01110': 5948, '11010': 6343, '00001': 4758, '10010': 5275, '01101': 5847, '11110': 7874, '10101': 5423, '111

In [3]:
# plot_histogram([ideal_counts, noisy_counts, corrected_counts], number_to_keep=5, legend=["ideal counts", "noisy counts", "corrected counts"], figsize=(20,20), filename="plot_sim.jpg")


In [4]:
num_qubits=len(list(noisy_counts.items())[0][0])

if 2**15<2**num_qubits:
    num_keep=2**10#num_qubits#2**10
else:
    num_keep=2**num_qubits
noisy_counts_orig=noisy_counts
print("num keep ", num_keep)
noisy_noise_est_dist=dist_ult.convert_to_first_col(noisy_noise_est_dist, col_label)
noisy_noise_est_dist=dist_ult.filter_counts(noisy_noise_est_dist, num_keep)
noisy_counts=dist_ult.filter_counts(noisy_counts, num_keep)
all_keys, noisy_noise_est_dist, noisy_counts=dist_ult.fill_counts_range(noisy_noise_est_dist, noisy_counts, num_keep, num_qubits)
print(noisy_counts)
print(noisy_noise_est_dist)

noisy_noise_est_vec=sorted(list(noisy_noise_est_dist.items()), key=lambda x: int(x[0], 2))
noisy_noise_est_vec=[elem[1] for elem in noisy_noise_est_vec]
counts_vec=sorted(list(noisy_counts.items()), key=lambda x: int(x[0], 2))
counts_vec=[elem[1] for elem in counts_vec]
# print(len(noisy_noise_est_vec))
# print(len(counts_vec))
quasi_corrected_counts=dist_ult.get_quasi_corrected(noisy_noise_est_vec, counts_vec)
quasi_corrected_counts=dist_ult.label_quasis(quasi_corrected_counts, all_keys)
# print(quasi_corrected_counts)
quasi_method=QuasiDistribution(quasi_corrected_counts)
quasi_corrected_counts=quasi_method.nearest_probability_distribution().binary_probabilities(num_qubits)
# print(quasi_corrected_counts)
# quasi_corrected_counts={bin(idx)[2::].zfill(num_qubits):val for idx, val in enumerate(quasi_corrected_counts)}



num keep  32
max exist  31
min exist  0
size before  32
{'00000': 4787, '00001': 4758, '00010': 4972, '00011': 5117, '00100': 4878, '00101': 4986, '00110': 5020, '00111': 5621, '01000': 5616, '01001': 5620, '01010': 5720, '01011': 6091, '01100': 5666, '01101': 5847, '01110': 5948, '01111': 7868, '10000': 5345, '10001': 5188, '10010': 5275, '10011': 5686, '10100': 5157, '10101': 5423, '10110': 5684, '10111': 6342, '11000': 6113, '11001': 6319, '11010': 6343, '11011': 7373, '11100': 6443, '11101': 7544, '11110': 7874, '11111': 19376}
{'00000': 0.13557, '00001': 0.06075, '00010': 0.059655, '00011': 0.03652, '00100': 0.05885, '00101': 0.03333, '00110': 0.03243, '00111': 0.020555, '01000': 0.05342, '01001': 0.031045, '01010': 0.02986, '01011': 0.01832, '01100': 0.02979, '01101': 0.017905, '01110': 0.01785, '01111': 0.01215, '10000': 0.058475, '10001': 0.03076, '10010': 0.032555, '10011': 0.019575, '10100': 0.036365, '10101': 0.02001, '10110': 0.01994, '10111': 0.01301, '11000': 0.027985, '1

In [5]:
fig=plot_histogram([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, bar_labels=True,
               legend=["ideal", "raw", "corrected"], title="Grover Search 5 Qubits Simulation")
fig.savefig("plot_grover5_counts_sim.jpg", bbox_inches="tight")

In [6]:
fig=plot_distribution([ideal_counts, noisy_counts_orig, quasi_corrected_counts], number_to_keep=5, bar_labels=True,
                  legend=["ideal", "raw", "corrected"], title="Grover Search 5 Qubits Simulation") #figsize=(15,15))
ax=fig.axes[0]
ax.set_ylabel("Probability")
fig.tight_layout()
fig.savefig("plot_grover5_dist_sim.jpg", bbox_inches="tight")